This Colab illustrates an example of mesh deformation.

In [0]:
import numpy as np
import tensorflow as tf
from matplotlib import collections
import matplotlib.pyplot as plt

from colabtools import adhoc_import
with adhoc_import.Google3():
  from tensorflow_graphics.deformation_energies import as_conformal_as_possible
  from tensorflow_graphics.transformation import quaternion

Helper functions to sample points along a line and to display results.

In [0]:
def _draw_lines_and_constaints(points_2d, edges, title=None, landmarks=None):
  """Draws 2d line segments and landmarks.

  Args:
    points_2d: A numpy array containing 2d points.
    edges: a vector of 2d tuples containing all the edges to draw. Each edge
      contains two indices refering to points stored in `points_2d`.
    title: Optional string defining the title of the plotted image.
    landmarks: Optional numpy array of shape [N,2] containing landmark points to
      render.
  """
  n_edges = len(edges)
  lines = [(points_2d[edges[edge_index][0], :],
            points_2d[edges[edge_index][1], :])
           for edge_index in range(n_edges)]
  lc = collections.LineCollection(lines)
  _, ax = plt.subplots()
  ax.axes.get_xaxis().set_visible(False)
  ax.axes.get_yaxis().set_visible(False)
  ax.add_collection(lc)
  ax.set_xlim((-2.0, 2.0))
  ax.set_ylim((-2.0, 2.0))

  if title is not None:
    ax.set_title(title)

  if landmarks is not None:
    radius = min(
        np.amax(points_2d[:, 0]) - np.amin(points_2d[:, 0]),
        np.amax(points_2d[:, 1]) - np.amin(points_2d[:, 1])) / 20.0
    for v in range(landmarks.shape[0]):
      circle = plt.Circle((landmarks[v, 0], landmarks[v, 1]), radius, color="r")
      ax.add_artist(circle)


def _sample_points_along_line_segment(start_point, endpoint, n_points):
  """Samples ordered and equally spaced points along a line segment.

  Args:
    start_point: A numpy array of shape [2] containing the 2d position of the
      start point of the line segment.
    endpoint: A numpy array of shape [2] containing the 2d position of the
      endpoint of the line segment.
    n_points: The number of points to generate along the line segment.

  Returns:
    A list of tuples contraining ordered and equally spaced 2d points
    distributed between `start_point` and `endpoint`.
  """
  edges = []
  step = (endpoint - start_point) / (n_points - 1)
  point_index = 0
  points = np.zeros((n_points, 2))
  for point_index in range(n_points - 1):
    points[point_index] = start_point + point_index * step
    edges.append((point_index, point_index + 1))
  points[n_points - 1] = start_point + (n_points - 1) * step
  return points, edges

In [0]:
###############
# UI controls #
###############
#@title Contraints on the deformed pose { vertical-output: false, run: "auto" }
constraint_1_y = 0.0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }
constraint_2_y = -1.0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }
constraint_3_y = 0  #@param { type: "slider", min: -1, max: 1 , step: 0.05 }

tf.reset_default_graph()

########################
# Problem construction #
########################
# Builds the rest pose.
start_point_line = np.array((-1.0, 0.0))
endpoint_line = np.array((1.0, 0.0))
n_samples = 20
vertices_rest_pose, connectivity = _sample_points_along_line_segment(
    start_point_line, endpoint_line, n_samples)
vertices_rest_pose = tf.convert_to_tensor(vertices_rest_pose)

# Initializes the deformed pose by adding contraints to the rest pose.
half_sample = n_samples / 2
constraints = np.array(((-1.0, constraint_1_y), (0.0, constraint_2_y),
                        (1.0, constraint_3_y)))
tf_constraints = [
    tf.constant((-1.0 + index, constraint), dtype=vertices_rest_pose.dtype)
    for index, constraint in enumerate((constraint_1_y, constraint_2_y,
                                        constraint_3_y))
]
free_variables_1 = tf.Variable(vertices_rest_pose[1:half_sample - 1])
free_variables_2 = tf.Variable(vertices_rest_pose[half_sample:-1])
vertices_deformed_pose = tf.concat(
    ((tf_constraints[0],), free_variables_1,
     (tf_constraints[1],), free_variables_2, (tf_constraints[2],)), 0)
rotations = tf.Variable(quaternion.from_euler(np.zeros((n_samples, 3))))

with tf.name_scope("initialize_variables"):
  init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

# Shows the rest pose and the initial deformed pose.
_draw_lines_and_constaints(
    vertices_rest_pose.eval(session=sess), connectivity, "Rest pose")
_draw_lines_and_constaints(
    vertices_deformed_pose.eval(session=sess), connectivity, "Constraints",
    constraints)

# Builds the deformation energy.
fill = tf.zeros(shape=(n_samples, 1), dtype=vertices_rest_pose.dtype)
vertices_rest_pose = tf.concat((vertices_rest_pose, fill), axis=-1)
vertices_deformed_pose = tf.concat((vertices_deformed_pose, fill), axis=-1)
energy = as_conformal_as_possible.energy(
    vertices_rest_pose, vertices_deformed_pose, rotations, connectivity)

################
# Optimization #
################
with tf.name_scope("optimizer"):
  optimizer = tf.contrib.opt.ScipyOptimizerInterface(
      energy, options={"maxiter": n_samples})

n_iterations = 20
for it in range(n_iterations):
  optimizer.minimize(sess)
  vertices_deformed_pose_ = sess.run(vertices_deformed_pose)

_draw_lines_and_constaints(
    vertices_deformed_pose.eval(session=sess)[:, 0:2], connectivity,
    "Deformed pose", constraints)